In [8]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.6 MB 1.7 MB/s eta 0:00:23
   ---------------------------------------- 0.2/38.6 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.5/38.6 MB 3.1 MB/s eta 0:00:13
    --------------------------------------- 0.8/38.6 MB 4.5 MB/s eta 0:00:09
   - -------------------------------------- 1.2/38.6 MB 5.3 MB/s eta 0:00:08
   - -------------------------------------- 1.8/38.6 MB 6.4 MB/s eta 0:00:06
   -- ------------------------------------- 2.0/38.6 MB 6.8 MB/s eta 0:00:06
   -- ------------------------------------- 2.4/38.6 MB 6.4 MB/s eta 0:00:06
   -- -----------------

# Face Detection Model
- Load the models

In [11]:
face_detection_model = './models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'
# load models using cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [ ]:
!dir images

In [ ]:
# consider sample image
img = cv2.imread('./images/Sachin Tendulkar/2200.jpg')
cv2.imshow('sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def helper(image_path):
    img = cv2.imread(image_path)
    # step-1: face detection
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])# consider the face with max confidence score
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # step-2: Feature Extraction or Embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
            
            return vectors
    return None


# apply helper function to all images and get face descriptors

In [19]:
data = dict(data=[],label=[])

In [21]:
folders = os.listdir('images')
for folder in folders:
    filenames = os.listdir('images/{}'.format(folder))
    for filename in filenames:
        try:

            vector = helper('./images/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extracted Sucessfully')
                
        except:
           pass

In [16]:
data.keys()

dict_keys(['data', 'label'])

In [17]:
pd.Series(data['label']).value_counts()

Series([], Name: count, dtype: int64)

In [18]:
# save the data
pickle.dump(data,open('data_face_features.pickle',mode='wb'))